In [1]:
%load_ext lab_black
%load_ext autotime
import pandas as pd
import numpy as np

time: 258 ms (started: 2022-09-15 23:30:53 -07:00)


The [20 Newsgroups dataset](http://qwone.com/~jason/20Newsgroups/). This treatment is based on the material in <https://umap-learn.readthedocs.io/en/latest/sparse.html>, which leaves the data sparse. Not all dimensionality reduction methods can handle sparse data, so it will be converted to a 3000D dense matrix via PCA. **Warning**: the truncated SVD will cause this notebook to take up a fair bit of RAM (around 11GB).

In [2]:
import sklearn.datasets
import sklearn.feature_extraction.text

ng20v = sklearn.datasets.fetch_20newsgroups_vectorized(subset="all")
ng20tfidf = sklearn.feature_extraction.text.TfidfTransformer(norm="l1").fit_transform(
    ng20v.data
)

time: 1.21 s (started: 2022-09-15 23:30:53 -07:00)


In [3]:
ng20tfidf.shape

(18846, 130107)

time: 7.13 ms (started: 2022-09-15 23:30:54 -07:00)


In [4]:
ng20tfidf

<18846x130107 sparse matrix of type '<class 'numpy.float64'>'
	with 2895521 stored elements in Compressed Sparse Row format>

time: 11.4 ms (started: 2022-09-15 23:30:54 -07:00)


In [5]:
import sklearn.decomposition

time: 33.4 ms (started: 2022-09-15 23:30:54 -07:00)


Apart from eating up a fair bit of RAM, this next step is also pretty slow (around ten minutes on my machine):

In [6]:
tsvd = sklearn.decomposition.TruncatedSVD(n_components=3000).fit(ng20tfidf)

time: 9min 18s (started: 2022-09-15 23:30:54 -07:00)


How much variance does 3000 components explain?

In [7]:
np.sum(tsvd.explained_variance_ratio_)

0.7099497001717341

time: 3.86 ms (started: 2022-09-15 23:40:13 -07:00)


71%? Not terrible.

In [8]:
data = tsvd.transform(ng20tfidf)

time: 6.23 s (started: 2022-09-15 23:40:13 -07:00)


In [9]:
ng20v.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

time: 3.91 ms (started: 2022-09-15 23:40:19 -07:00)


In [10]:
ng20v.target

array([17,  7, 10, ..., 10, 18,  9])

time: 4.46 ms (started: 2022-09-15 23:40:19 -07:00)


In [11]:
description = pd.Series(
    list(map(ng20v.target_names.__getitem__, ng20v.target.astype(int))),
    name="description",
)

time: 6.63 ms (started: 2022-09-15 23:40:19 -07:00)


In [12]:
description

0        talk.politics.mideast
1                    rec.autos
2             rec.sport.hockey
3             rec.sport.hockey
4                    rec.autos
                 ...          
18841       talk.politics.misc
18842       talk.politics.guns
18843         rec.sport.hockey
18844       talk.politics.misc
18845       rec.sport.baseball
Name: description, Length: 18846, dtype: object

time: 12.5 ms (started: 2022-09-15 23:40:19 -07:00)


In [13]:
target = pd.concat([pd.Series(ng20v.target, name="class"), description], axis=1)

time: 2.93 ms (started: 2022-09-15 23:40:19 -07:00)


In [14]:
target

,class,description
0,17,talk.politics.mideast
1,7,rec.autos
2,10,rec.sport.hockey
3,10,rec.sport.hockey
4,7,rec.autos
...,...,...
18841,18,talk.politics.misc
18842,16,talk.politics.guns
18843,10,rec.sport.hockey
18844,18,talk.politics.misc


time: 13.9 ms (started: 2022-09-15 23:40:19 -07:00)


## Pipeline

In [15]:
from drnb.io.pipeline import create_default_pipeline

data_result = create_default_pipeline(check_for_duplicates=True).run(
    "ng20",
    data=data,
    target=target,
    tags=["highdim"],
    url="http://qwone.com/~jason/20Newsgroups/",
    verbose=True,
)

[23:40:23] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=440513;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=722540;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#392\392]8;;\

           INFO     Initial data shape: (18846, 3000)                                                ]8;id=850243;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=102540;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#80\80]8;;\

           INFO     Keeping all columns                                                            ]8;id=156241;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=949413;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     Data shape after filtering columns: (18846, 3000)                               ]8;id=372940;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=852381;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#151\151]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=206447;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=828616;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#135\135]8;;\

...

           INFO     Writing csv format to triplets/ng20.5.1337.l2.csv                               ]8;id=718791;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=510842;file:///home/james/dev/drnb/src/drnb/io/__init__.py#221\221]8;;\

           INFO     Writing numpy format to triplets/ng20.5.1337.idx.npy                            ]8;id=655768;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=490961;file:///home/james/dev/drnb/src/drnb/io/__init__.py#244\244]8;;\

           INFO     Writing numpy format to triplets/ng20.5.1337.l2.npy                             ]8;id=529146;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=331421;file:///home/james/dev/drnb/src/drnb/io/__init__.py#244\244]8;;\

           INFO     Writing pipeline result for ng20                                                ]8;id=650454;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=255563;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#128\128]8;;\

time: 1min 3s (started: 2022-09-15 23:40:19 -07:00)
